In [1]:
import os 
%pwd

'd:\\pythonProjects\\Agentic_AI\\research'

In [2]:
os.chdir('../')

In [3]:
%pwd 

'd:\\pythonProjects\\Agentic_AI'

In [15]:
# entity
from dataclasses import dataclass
from pathlib import Path  

@dataclass
class ModelTrainerConfig:
    root_dir: Path 
    data_path: Path 
    model_ckpt: Path 
    num_train_epochs: int 
    warmup_steps: int 
    per_device_train_batch_size: int 
    weight_decay: float 
    logging_steps: int 
    eval_strategy: str 
    eval_steps: int 
    save_steps: float 
    gradient_accumulation_steps: int 


In [16]:
# configuration
from src.textSummerizer.constants import * 
from src.textSummerizer.utils.common import read_yaml, create_directory
from datasets import load_from_disk


class ConfgurationManager: 
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 param_filepath=PARAMS_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(param_filepath)

        create_directory([self.config.artifacts_root])

    def get_model_trainer_config(self)-> ModelTrainerConfig:
        config=self.config.model_trainer
        params=self.params.TrainingArguments
        create_directory([config.root_dir])

        model_trainer_config=ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt=config.model_ckpt,
            num_train_epochs=params.num_train_epochs,
            warmup_steps=params.warmup_steps,
            per_device_train_batch_size=params.per_device_train_batch_size,
            weight_decay=params.weight_decay,
            logging_steps=params.logging_steps,
            eval_steps=params.eval_steps,
            save_steps=params.save_steps,
            gradient_accumulation_steps=params.gradient_accumulation_steps,
            eval_strategy=params.evaluation_strategy

        )
        return model_trainer_config       

In [ ]:
# components
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import TrainingArguments, Trainer 
from transformers import DataCollatorForSeq2Seq
import torch

class ModelTrainer: 
    def __init__(self, config:ModelTrainerConfig):
        self.config=config

    def train(self):
        device='cuda' if torch.cuda.is_available() else 'cpu'
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus=AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator=DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

        # load the data
        dataset_samsum_pt=load_from_disk(self.config.data_path)
        
        trainer_args=TrainingArguments(
            output_dir=self.config.root_dir, num_train_epochs=1, warmup_steps=500,
            per_device_train_batch_size=1, per_device_eval_batch_size=1,
            weight_decay=0.01, logging_steps=10,
            eval_strategy='steps', eval_steps=500, save_steps=1e6,
            gradient_accumulation_steps=16
        )

        trainer=Trainer(model=model_pegasus, args=trainer_args,
                        tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                        train_dataset=dataset_samsum_pt['test'],
                        eval_dataset=dataset_samsum_pt['validation']
                        )
        trainer.train()
        # save model
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir,'pegasus-samsum-model'))
        # save tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir,'tokenizer'))

In [20]:
config=ConfgurationManager()
model_trainer_config=config.get_model_trainer_config()
model_trainer_config=ModelTrainer(config=model_trainer_config)
model_trainer_config.train()

[2025-04-19 05:42:15,115: INFO: common: Yaml file config\config.yaml loaded successfully]
[2025-04-19 05:42:15,118: INFO: common: Yaml file params.yaml loaded successfully]
[2025-04-19 05:42:15,119: INFO: common: created directory at: artifacts]
[2025-04-19 05:42:15,121: INFO: common: created directory at: artifacts/model_trainer]


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\26amr\AppData\Local\Temp\ipykernel_38844\2479392869.py:28: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer=Trainer(model=model_pegasus, args=trainer_args,


Step,Training Loss,Validation Loss


d:\pythonProjects\Agentic_AI\venv\Lib\site-packages\transformers\modeling_utils.py:3339: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


AttributeError: 'PegasusForConditionalGeneration' object has no attribute 'save_pretrain'